In [1]:
# Importamos las bibliotecas necesarias para el funcionamiento del script.
# `requests`: Se utiliza para realizar solicitudes HTTP y obtener el contenido de páginas web.
# `BeautifulSoup` (importado como `bs`): Es una herramienta poderosa para parsear y manipular documentos HTML/XML.
# `pandas` (importado como `pd`): Se utiliza para manejar datos tabulares y crear DataFrames.
# `numpy` (importado como `np`): Proporciona soporte para valores faltantes (`np.nan`) y operaciones numéricas.
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [2]:
# Realizamos una solicitud GET a la página que contiene la lista de libros de Skyrim.
# La respuesta contiene el contenido HTML de la página.
html = requests.get('https://en.uesp.net/wiki/Skyrim:Books')

# Parseamos el contenido HTML utilizando BeautifulSoup con el parser 'html.parser'.
# Esto nos permite trabajar con el HTML de manera más estructurada y fácil de manipular.
page = bs(html.content, 'html.parser')

In [3]:
# Buscamos todos los elementos `<tr>` dentro del HTML de la página.
# Estos elementos representan las filas de la tabla que contiene la información de los libros.
bookList = page.find_all('tr')

# Eliminamos las primeras 7 filas, ya que contienen información no relevante (encabezados y otros detalles).
bookList = bookList[7:]

In [4]:
# Creamos una lista vacía llamada `bookTitles` para almacenar los títulos de los libros.
bookTitles = []

# Iteramos sobre cada fila en `bookList` para extraer los títulos.
for book in bookList:
    # Verificamos si la fila contiene una etiqueta `<b>` (negrita), que típicamente contiene el título del libro.
    if book.b and book.b.text.strip():  # Aseguramos que el texto no esté vacío después de eliminar espacios.
        bookTitles.append(book.b.text.strip())  # Agregamos el título a la lista.
    else:
        bookTitles.append(np.nan)  # Si no hay título, agregamos `np.nan` para indicar un valor faltante.

In [5]:
# Creamos una lista vacía llamada `bookCosts` para almacenar los costos de los libros.
bookCosts = []

# Iteramos sobre cada fila en `bookList` para extraer los costos.
for book in bookList:
    cells = book.find_all('td')  # Encontramos todas las celdas (`<td>`) de la fila.
    
    if len(cells) > 2:  # Aseguramos que exista al menos una columna para el costo.
        bookCosts.append(cells[2].text.strip())  # El costo está en la tercera columna (índice 2).
    else:
        bookCosts.append(np.nan)  # Si no hay suficientes columnas, agregamos `np.nan`.

In [6]:
# Creamos una lista vacía llamada `bookAuthors` para almacenar los autores de los libros.
bookAuthors = []

# Iteramos sobre cada fila en `bookList` para extraer los autores.
for book in bookList:
    cells = book.find_all('td')  # Encontramos todas las celdas (`<td>`) de la fila.
    
    if len(cells) > 3:  # Aseguramos que exista al menos una columna para el autor.
        bookAuthors.append(cells[3].text.strip())  # El autor está en la cuarta columna (índice 3).
    else:
        bookAuthors.append(np.nan)  # Si no hay suficientes columnas, agregamos `np.nan`.

In [7]:
# Creamos una lista vacía llamada `bookDescriptions` para almacenar las descripciones de los libros.
bookDescriptions = []

# Iteramos sobre cada fila en `bookList` para extraer las descripciones.
for book in bookList:
    cells = book.find_all('td')  # Encontramos todas las celdas (`<td>`) de la fila.
    
    if len(cells) > 4:  # Aseguramos que exista al menos una columna para la descripción.
        bookDescriptions.append(cells[4].text.strip())  # La descripción está en la quinta columna (índice 4).
    else:
        bookDescriptions.append(np.nan)  # Si no hay suficientes columnas, agregamos `np.nan`.

In [8]:
# Creamos una lista vacía llamada `bookSkill` para almacenar información sobre si un libro es de habilidades.
bookSkill = []

# Iteramos sobre cada fila en `bookList` para determinar si un libro es de habilidades.
for book in bookList:
    cells = book.find_all('td')  # Encontramos todas las celdas (`<td>`) de la fila.
    
    if len(cells) == 6:  # Los libros de habilidades tienen exactamente 6 columnas.
        bookSkill.append(cells[5].text.strip())  # La información de habilidad está en la sexta columna (índice 5).
    else:
        bookSkill.append(np.nan)  # Si no hay suficientes columnas, agregamos `np.nan`.

In [9]:
# Verificamos que todas las listas tengan la misma longitud antes de crear el DataFrame.
print(f"{len(bookTitles)} records in 'bookTitles'")
print(f"{len(bookCosts)} records in 'bookCosts'")
print(f"{len(bookAuthors)} records in 'bookAuthors'")
print(f"{len(bookDescriptions)} records in 'bookDescriptions'")
print(f"{len(bookSkill)} records in 'bookSkill'")

533 records in 'bookTitles'
533 records in 'bookCosts'
533 records in 'bookAuthors'
533 records in 'bookDescriptions'
533 records in 'bookSkill'


In [10]:
# Creamos un DataFrame de Pandas combinando todas las listas extraídas.
df = pd.DataFrame({
    'Title': bookTitles,
    'Cost': bookCosts,
    'Author': bookAuthors,
    'Description': bookDescriptions,
    'Skill': bookSkill
})

# Inspeccionamos las primeras filas del DataFrame para verificar su estructura.
print(df.head())

                           Title Cost            Author  \
0   16 Accords of Madness, v. VI   25                     
1   16 Accords of Madness, v. IX   25                     
2  16 Accords of Madness, v. XII   25                     
3         2920, Morning Star, v1   50  Carlovac Townway   
4           2920, Sun's Dawn, v2   60  Carlovac Townway   

                                         Description                  Skill  
0                                     Hircine's Tale                         
1                                    Vaermina's Tale                         
2                                    Malacath's Tale                         
3  Volume 1 of a historical series about Vivec an...  One-handed skill book  
4  Volume 2 of a historical series about Vivec an...    Illusion skill book  


In [11]:
# Convertimos los valores de la columna 'Skill' a números para facilitar el análisis.
# Si el texto contiene "skill book", asignamos 1; de lo contrario, asignamos 0.
df['Skill'] = df['Skill'].apply(lambda x: 1 if isinstance(x, str) and "skill book" in x.lower() else 0)

# Inspeccionamos las primeras filas del DataFrame para verificar los cambios en la columna 'Skill'.
print(df.head(5))

                           Title Cost            Author  \
0   16 Accords of Madness, v. VI   25                     
1   16 Accords of Madness, v. IX   25                     
2  16 Accords of Madness, v. XII   25                     
3         2920, Morning Star, v1   50  Carlovac Townway   
4           2920, Sun's Dawn, v2   60  Carlovac Townway   

                                         Description  Skill  
0                                     Hircine's Tale      0  
1                                    Vaermina's Tale      0  
2                                    Malacath's Tale      0  
3  Volume 1 of a historical series about Vivec an...      1  
4  Volume 2 of a historical series about Vivec an...      1  


In [12]:
# Guardamos el DataFrame en un archivo CSV llamado 'skyrim_books.csv'.
# Usamos `index=False` para evitar guardar la columna de índice.
df.to_csv('skyrim_books.csv', index=False)